In [2]:
#from minnie65 import m65, ingest, plotting
#from minnie65.utils.em_2p import func, utils
import datajoint as dj
import numpy as np
m65 = dj.create_virtual_module('m65', 'microns_minnie65_01')



Connecting celiib@10.28.0.34:3306


In [3]:
from ipywidgets import widgets, HBox, VBox, Label
from IPython.display import HTML, display, clear_output
import urllib
import json
import re

In [4]:
def html_to_json(url_string, return_parsed_url=False, fragment_prefix='!'):
    # Parse neuromancer url to logically separate the json state dict from the rest of it.
    full_url_parsed = urllib.parse.urlparse(url_string)
    # Decode percent-encoding in url, and skip "!" from beginning of string.
    decoded_fragment = urllib.parse.unquote(full_url_parsed.fragment)
    if decoded_fragment.startswith(fragment_prefix):
        decoded_fragment = decoded_fragment[1:]
    # Load the json state dict string into a python dictionary.
    json_state_dict = json.loads(decoded_fragment)

    if return_parsed_url:
        return json_state_dict, full_url_parsed
    else:
        return json_state_dict


def add_segments(provided_link, segments, seg_name='seg', overwrite=True, color=None):
    json_data, parsed_url = html_to_json(provided_link, return_parsed_url=True)
    seg_strings = []
    for seg in segments:
        seg_strings.append(seg.astype(np.str))
    segmentation_layer = list(filter(lambda d: np.logical_and(d['type'] == 'segmentation', d['name'] == seg_name), json_data['layers']))
    if len(segmentation_layer) == 0:
        print('segmentation layer does not exist... creating it')
        json_data['layers'].append({
        "source": "precomputed://gs://microns-seunglab/minnie65/seg_minnie65_0",
        "type": "segmentation",
        "colorSeed": 2940450712,
        "segments": [],
        "skeletonRendering": {
        "mode2d": "lines_and_points",
        "mode3d": "lines"
          },
       "name": f"{seg_name}"
        })
        segmentation_layer = list(filter(lambda d: np.logical_and(d['type'] == 'segmentation', d['name'] == seg_name), json_data['layers']))
    if re.search('segments',json.dumps(json_data)) is None:
        segmentation_layer[0].update({'segments':[]})
    if overwrite:
        segmentation_layer[0]['segments'] = seg_strings
    else:
        segmentation_layer[0]['segments'].extend(seg_strings)
    if color is not None:
        if re.search('segmentColors',json.dumps(json_data)) is None:
            segmentation_layer[0].update({'segmentColors':{}})
        color_dict = {}
        for seg in segments:
            color_dict.update({str(seg):color})
        segmentation_layer[0]['segmentColors'] = color_dict
            
    return urllib.parse.urlunparse([parsed_url.scheme, parsed_url.netloc, parsed_url.path, parsed_url.params, parsed_url.query, '!'+ urllib.parse.quote(json.dumps(json_data))])

In [5]:
@m65.schema
class AllenMultiSomas(dj.Manual):
    definition = """
    # Segment IDs of core segments with multiple somas identified by the Allen Institute.
    segment_id           : bigint unsigned              # segment id unique within each Segmentation, should be mirrored in Segment table.
    ---
    status=NULL : int unsigned                          # verification status (NULL=not checked, 0=ignore, 1=partial, 2=complete)
    """

    class Centroids(dj.Part):
        definition = """
        # Manually added centroids for AllenMultiSoma table
        -> AllenMultiSomas
        soma_id    : smallint unsigned              # soma id of a subsegment
        ---
        centroid_x  : int unsigned                        # x coordinate of centroid in EM voxels
        centroid_y  : int unsigned                        # y coordinate of centroid in EM voxels
        centroid_z  : int unsigned                        # z coordinate of centroid in EM voxels
        """

In [50]:
def updateAllenMultiSomaCentroid(provided_link, random=False):
    if random:
        unchecked = (AllenMultiSomas() & 'status is NULL').fetch('segment_id')
        seg_id = np.random.permutation(unchecked)[0]
            
    # fields
        segment_field = widgets.Text(str(seg_id), continuous_update=True)
    else:
        segment_field = widgets.Text('Enter Segment ID here', continuous_update=True)
    centroid_field = widgets.Text('Enter EM Centroid here', continuous_update=True)
    
    # buttons
    ng_button = widgets.Button(description="Print NG Link")    
    def on_ng_button_clicked(ng_button):
        with_seg = add_segments(provided_link, np.array([eval(segment_field.value)]), 'seg')
        display(HTML(f'<a href="{with_seg}">Neuroglancer link for seg: {segment_field.value}</a>'))
        print(HTML(f'<a href="{with_seg}">Neuroglancer link for seg: {segment_field.value}</a>'))
        
    
    insert_button = widgets.Button(description="Insert")
    def on_insert_button_clicked(insert_button):
        cen_x, cen_y, cen_z = np.array(eval(centroid_field.value))
        n_soma = len(AllenMultiSomas.Centroids & f'segment_id={eval(segment_field.value)}')
        AllenMultiSomas.Centroids.insert1([eval(segment_field.value), n_soma+1, cen_x, cen_y, cen_z], allow_direct_insert=True)
        (AllenMultiSomas & f'segment_id = {eval(segment_field.value)}')._update('status', 1)
        print(f'Successful Centroid Insert: seg_id: {eval(segment_field.value)}, soma_id: {n_soma+1}, centroid: {cen_x, cen_y, cen_z}, status=1 (Partial)')
#         print(f'Successful Update: seg_id: {eval(segment_field.value)}, status=1 (Partial)')
    
    null_button = widgets.Button(description='NULL')
    def on_null_button_clicked(null_button):
        (AllenMultiSomas & f'segment_id = {eval(segment_field.value)}')._update('status', np.nan)
        print(f'Successful Update: seg_id: {eval(segment_field.value)}, status=NULL (not checked)')
    
    ignore_button = widgets.Button(description='Ignore')
    def on_ignore_button_clicked(ignore_button):
        (AllenMultiSomas & f'segment_id = {eval(segment_field.value)}')._update('status', 0)
        print(f'Successful Update: seg_id: {eval(segment_field.value)}, status=0 (Ignore)')
        
    ignore_button_3 = widgets.Button(description='Somas_Merged')
    def on_ignore_button_3_clicked(ignore_button_3):
        (AllenMultiSomas & f'segment_id = {eval(segment_field.value)}')._update('status', 3)
        print(f'Successful Update: seg_id: {eval(segment_field.value)}, status=3 (Somas_Merged)')
    
    ignore_button_4 = widgets.Button(description='Glia_Merged')
    def on_ignore_button_4_clicked(ignore_button_4):
        (AllenMultiSomas & f'segment_id = {eval(segment_field.value)}')._update('status', 4)
        print(f'Successful Update: seg_id: {eval(segment_field.value)}, status=4 (Glia_Merged)')
        
    ignore_button_5 = widgets.Button(description='Glia_Merged_onto_Soma')
    def on_ignore_button_5_clicked(ignore_button_5):
        (AllenMultiSomas & f'segment_id = {eval(segment_field.value)}')._update('status', 5)
        print(f'Successful Update: seg_id: {eval(segment_field.value)}, status=5 (Glia_Merged_onto_Soma)')
        
        
    partial_button = widgets.Button(description="Partial")
    def on_partial_button_clicked(partial_button):
        (AllenMultiSomas & f'segment_id = {eval(segment_field.value)}')._update('status', 1)
        print(f'Successful Update: seg_id: {eval(segment_field.value)}, status=1 (Partial)')
    
    
    complete_button = widgets.Button(description="Complete_2")
    def on_complete_button_clicked(complete_button):
        (AllenMultiSomas & f'segment_id = {eval(segment_field.value)}')._update('status', 2)
        print(f'Successful Update: seg_id: {eval(segment_field.value)}, status=2 (Complete)')
        
    complete_button_1 = widgets.Button(description="Complete (1)")
    def on_complete_button_1_clicked(complete_button_1):
        (AllenMultiSomas & f'segment_id = {eval(segment_field.value)}')._update('status', 6)
        print(f'Successful Update: seg_id: {eval(segment_field.value)}, status=6 (Complete_1)')
    
    # display
    display(HBox([Label(value = r'$\Large \text{Update Segment ID:} $'), segment_field, ng_button]))
    display(HBox([Label(value = r'$\Large \text{Update Centroid:}$'), centroid_field, insert_button]))
    display(HBox([Label(value = r'$\Large \text{Update Status:}$'), null_button, ignore_button,
                          ignore_button_3,ignore_button_4,ignore_button_5,
                          partial_button, complete_button,complete_button_1]))
    
#     plot preview (too slow right now, might be better with higher decimation like 0.05)
#     mesh = (m65.Decimation() & f'segment_id={seg_id}').fetch1('mesh')
#     @plotting.config_dark
#     def plot_mesh(mesh):
#         plotting.add_mesh(mesh.vertices[:,[0,2,1]], mesh.faces, color='green') #need to fix the orientation still
#     plot_mesh(mesh)
    
    ng_button.on_click(on_ng_button_clicked)
    insert_button.on_click(on_insert_button_clicked)
    null_button.on_click(on_null_button_clicked)
    ignore_button.on_click(on_ignore_button_clicked)
    ignore_button_3.on_click(on_ignore_button_3_clicked)
    ignore_button_4.on_click(on_ignore_button_4_clicked)
    ignore_button_5.on_click(on_ignore_button_5_clicked)
    partial_button.on_click(on_partial_button_clicked)
    complete_button.on_click(on_complete_button_clicked)
    complete_button_1.on_click(on_complete_button_1_clicked)

In [51]:
provided_link = 'https://neuromancer-seung-import.appspot.com/#!%7B%22layers%22:%5B%7B%22source%22:%22precomputed://gs://microns-seunglab/minnie_v4/alignment/fine/sergiy_multimodel_v1/vector_fixer30_faster_v01/image_stitch_multi_block_v1%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22image%22%7D%2C%7B%22source%22:%22precomputed://gs://microns-seunglab/minnie65/single_sections%22%2C%22type%22:%22image%22%2C%22blend%22:%22default%22%2C%22shaderControls%22:%7B%7D%2C%22name%22:%22mip1%22%2C%22visible%22:false%7D%2C%7B%22source%22:%22precomputed://gs://microns-seunglab/minnie65/seg_minnie65_0%22%2C%22type%22:%22segmentation%22%2C%22skeletonRendering%22:%7B%22mode2d%22:%22lines_and_points%22%2C%22mode3d%22:%22lines%22%7D%2C%22name%22:%22seg%22%7D%5D%2C%22navigation%22:%7B%22pose%22:%7B%22position%22:%7B%22voxelSize%22:%5B4%2C4%2C40%5D%2C%22voxelCoordinates%22:%5B251075.296875%2C232138.1875%2C20365.94921875%5D%7D%2C%22orientation%22:%5B0%2C-0.7071067690849304%2C0%2C0.7071067690849304%5D%7D%2C%22zoomFactor%22:55.353050124031554%7D%2C%22perspectiveZoom%22:23722.467126512438%2C%22showSlices%22:false%2C%22gpuMemoryLimit%22:2500000000%2C%22systemMemoryLimit%22:2500000000%2C%22selectedLayer%22:%7B%22layer%22:%22seg%22%2C%22visible%22:true%7D%2C%22layout%22:%7B%22type%22:%224panel%22%2C%22orthographicProjection%22:true%7D%7D'

In [55]:
updateAllenMultiSomaCentroid(provided_link, random=True)

In [ ]:
"""
Shift drag to get to center
spacebar to get the soma center
Click copy position to clipboard in the top right
Paste in update centroid
Hit partial
"""

In [56]:
current_seg_id = 92345990027434104
m65.AllenMultiSomas & dict(segment_id=current_seg_id)

"segment_id segment id unique within each Segmentation, should be mirrored in Segment table.","status verification status (NULL=not checked, 0=ignore, 1=partial, 2=complete)"
92345990027434104,2


In [57]:
m65.AllenMultiSomas.Centroids()  & dict(segment_id=current_seg_id) #& "status=2"

"segment_id segment id unique within each Segmentation, should be mirrored in Segment table.",soma_id soma id of a subsegment,centroid_x x coordinate of centroid in EM voxels,centroid_y y coordinate of centroid in EM voxels,centroid_z z coordinate of centroid in EM voxels
92345990027434104,1,211786,252549,25004
92345990027434104,2,212979,282832,24652
92345990027434104,3,240359,280983,24745
92345990027434104,4,237516,252794,23318


In [20]:
m65.AllenMultiSomas.describe()

# Segment IDs of core segments with multiple somas identified by the Allen Institute.
segment_id           : bigint unsigned              # segment id unique within each Segmentation, should be mirrored in Segment table.
---
status=null          : int unsigned                 # verification status (NULL=not checked, 0=ignore, 1=partial, 2=complete)



'# Segment IDs of core segments with multiple somas identified by the Allen Institute.\nsegment_id           : bigint unsigned              # segment id unique within each Segmentation, should be mirrored in Segment table.\n---\nstatus=null          : int unsigned                 # verification status (NULL=not checked, 0=ignore, 1=partial, 2=complete)\n'

In [ ]:
"""
Different classes you would want to have:
3: somas_merged
4: glia_merged
5: glia_merged_onto_soma

"""